<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Face%20Mask%20Detection/Face_Mask_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 99% 327M/330M [00:02<00:00, 154MB/s]
100% 330M/330M [00:02<00:00, 153MB/s]


In [ ]:
!unzip --qq face-mask-12k-images-dataset.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [ ]:
batch_size = 12
epoch = 10
lr = 0.001
width = height = 178

In [ ]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
)

train_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Train',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Validation',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

test_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Test',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [ ]:
ResNet = tf.keras.applications.ResNet50V2(include_top=False, pooling='avg', weights='imagenet',
input_shape=(width, height, 3))

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.Sequential([
        ResNet,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(2, activation='sigmoid'),              
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'],
              )

In [ ]:
model.fit(train_data,
          steps_per_epoch = train_data.samples/batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples/batch_size,
          epochs = 10,
          )

Epoch 1/10
833/833 [==============================] - 237s 237ms/step - loss: 0.1263 - accuracy: 0.9596 - val_loss: 6.9175 - val_accuracy: 0.7425
Epoch 2/10
833/833 [==============================] - 195s 233ms/step - loss: 0.0677 - accuracy: 0.9808 - val_loss: 0.0297 - val_accuracy: 0.9912
Epoch 3/10
833/833 [==============================] - 195s 233ms/step - loss: 0.0548 - accuracy: 0.9840 - val_loss: 1.0115 - val_accuracy: 0.7412
Epoch 4/10
833/833 [==============================] - 195s 234ms/step - loss: 0.0512 - accuracy: 0.9855 - val_loss: 0.0461 - val_accuracy: 0.9862
Epoch 5/10
833/833 [==============================] - 196s 236ms/step - loss: 0.0353 - accuracy: 0.9906 - val_loss: 0.0102 - val_accuracy: 0.9962
Epoch 6/10
833/833 [==============================] - 196s 236ms/step - loss: 0.0479 - accuracy: 0.9845 - val_loss: 0.0208 - val_accuracy: 0.9887
Epoch 7/10
833/833 [==============================] - 196s 236ms/step - loss: 0.0355 - accuracy: 0.9887 - val_loss: 0.0119 -

In [ ]:
model.evaluate(test_data)

83/83 [==============================] - 6s 77ms/step - loss: 0.0333 - accuracy: 0.9889


[0.03331812471151352, 0.9889112710952759]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('FaceMask.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[241 242]
 [249 260]]
Classification Report
              precision    recall  f1-score   support

    WithMask       0.49      0.50      0.50       483
 WithoutMask       0.52      0.51      0.51       509

    accuracy                           0.51       992
   macro avg       0.50      0.50      0.50       992
weighted avg       0.51      0.51      0.51       992

